In [1]:
import psutil
import os
from diffusers import AutoPipelineForText2Image
import torch

model_id = "stabilityai/sdxl-turbo"
# model_id = "stabilityai/sd-turbo"
pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")


result_dir = f"results/{model_id.split('/')[-1]}"
os.makedirs(result_dir, exist_ok=True)

/workspace/miniconda/envs/img/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
text_encoder/config.json: 100%|██████████| 565/565 [00:00<00:00, 2.12MB/s]

scheduler/scheduler_config.json: 100%|██████████| 459/459 [00:00<00:00, 1.53MB/s]

text_encoder_2/config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

text_encoder_2/config.json: 100%|██████████| 575/575 [00:00<00:00, 54.2kB/s]


tokenizer/tokenizer_config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]
tokenizer/tokenizer_config.json: 100%|██████████| 704/704 [00:00<00:00, 84.2kB/s]
tokenizer/merges.tx

In [ ]:
text_01 = """
Title: The Little Garden
In a small village, there was a little garden. It was filled with colorful plants and fragrant flowers. The villagers loved to visit the garden to enjoy the beauty and fresh air.

After a storm, the villagers came together to fix the garden. They worked hard to repair the damaged plants and flowers.

The villagers felt proud and happy after they fixed the garden. They were grateful for their community coming together to restore the beauty of their little village.
"""

text_02 = """
Title: The Science Mystery"What is the purpose of science?" asked Ms. Johnson. "To discover new things and make our lives better," replied Alex.

Alex works hard and conducts experiments. One day, he discovers a new way to recycle plastic waste.

Alex felt excited and proud. "I can't wait to share this with everyone!" he exclaimed.
"""

text_03 = """
Title: The Elephant's Friend
In the heart of the jungle, there lived a majestic elephant named Ellie. Ellie was known for her grace and strength, but what set her apart from other elephants was her kind and gentle nature. She was not only a symbol of strength but also a symbol of compassion.

Ellie and her friends were enjoying a peaceful day in the jungle when they heard the sound of heavy footsteps. They looked around and saw a group of poachers entering their territory. Ellie's heart raced with fear as she thought about the danger they were in. She immediately called for her friends and together they hid behind a large tree, ready to defend themselves if necessary.

Ellie and her friends were ready to defend themselves against the poachers. They stood behind the tree, watching the poachers carefully. When they saw an opportunity, they quickly ran towards the poachers, using their strength and agility to chase them away. The poachers were surprised by the elephants' bravery and quickly retreated.
"""


t1 = "In a small village, a cherished garden faced storm damage; however, united efforts of the villagers restored its beauty, fostering pride and gratitude within the community."
t2 = "Ms. Johnson asks about the purpose of science; Alex, after diligent work, excitedly discovers a novel plastic recycling method, eager to share it with everyone."
t3 = "In the jungle, the compassionate elephant Ellie and her friends bravely thwart poachers, using their strength and agility to protect their territory and drive the intruders away."



text_01 = text_01.strip().split("\n\n") + [text_01, t1]
text_02 = text_02.strip().split("\n\n") + [text_02, t2]
text_03 = text_03.strip().split("\n\n") + [text_03, t3]

In [ ]:
import time
total_time, num = 0, 0
step = 2
for prompt in text_01+text_02+text_03:
    t0 = time.perf_counter()
    image = pipe(prompt=f"Draw below story {prompt} and write your story on the image", num_inference_steps=step, guidance_scale=0).images[0]
    total_time += (time.perf_counter() - t0)
    image.save(f"{result_dir}/{num+1:02}_{step}.png")
    num += 1

avg_time = total_time / num if num else 0
print(f"Avg time: {avg_time:.2f} seconds")